In [1]:
import cv2
import torch
from pathlib import Path
from ultralytics.nn.autobackend import AutoBackend
from ultralytics.yolo.utils.ops import non_max_suppression, scale_boxes
from ultralytics.yolo.data.dataloaders.stream_loaders import LoadImages, LoadStreams
from ultralytics.yolo.utils.plotting import Annotator, colors

# model config
half = False
img_sz = [320, 320]
device = 'cpu'
classes = list(range(80))
conf_thres = 0.5
iou_thres = 0.5
max_det = 1000
line_thickness = 2
agnostic_nms = False

f:\python\EveryDayCoding\ComputerVision\ObjectDetection\YOLO\Quantize\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ultralytics import YOLO

In [3]:
from ultralytics.nn.autobackend import AutoBackend
import onnxruntime
import numpy

In [4]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Export the model to ONNX format
model.export(format='onnx')  # creates 'yolov8n.onnx'
model.export(format='openvino')  # creates 'yolov8n_openvino_model/'

# Load the exported OpenVINO model
ov_model = YOLO('yolov8n_openvino_model/')

# Load the exported ONNX model
onnx_model = YOLO('yolov8n.onnx')

Ultralytics YOLOv8.0.75  Python-3.11.8 torch-2.2.2+cpu CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from yolov8n.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success  3.9s, saved as yolov8n.onnx (12.2 MB)

Export complete (15.0s)
Results saved to F:\python\EveryDayCoding\ComputerVision\ObjectDetection\YOLO\Quantize
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640 
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml 
Visualize:       https://netron.app
Ultralytics YOLOv8.0.75  Python-3.11.8 torch-2.2.2+cpu CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from yolov8n.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export succes

In [82]:
sess  = onnxruntime.InferenceSession('yolov8n.onnx', providers=onnxruntime.get_available_providers())
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

In [6]:
source = './Video/demo.mp4'
# model = YOLO('yolov8n.onnx', task='detect')
# model = YOLO('yolov8n.pt', task='detect')

In [7]:
model.half = True
model.imgsz=[640,640]
model.int8 = True

In [8]:
import time 

In [9]:
capture = cv2.VideoCapture(source)
# used to record the time when we processed last frame 
prev_frame_time = 0
  
# used to record the time at which we processed current frame 
new_frame_time = 0
# capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
# capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 320)

while (capture.isOpened()):
    ret, frame = capture.read() 
    frame = cv2.resize(frame, (640, 640))
    results = ov_model(frame)[0]
    
    if ret == True: 
        for box in results.boxes:
            class_name = results.names[box.cls[0].item()] # Class name
            cords = box.xyxy[0].tolist()
            x1, y1, x2, y2 = (round(x) for x in cords)
            x, y, w, h = x1, y1, x2 - x1, y2 - y1    
            # confidence = round(box.conf[0].item(), 2)
            # print(confidence)
            cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,255),3)
            
        new_frame_time = time.time() 
        fps = 1/(new_frame_time-prev_frame_time) 
        prev_frame_time = new_frame_time 
        fps = int(fps) 
        fps = str(fps)
        cv2.putText(frame, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX , 3, (100, 255, 0), 3, cv2.LINE_AA) 
        cv2.imshow('Frame', frame) 
        
        if cv2.waitKey(25) in (ord('q'), ord('Q')):
            break
    else: 
        break
  
# After the loop release the capture object 
capture.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

Loading yolov8n_openvino_model for OpenVINO inference...

0: 640x640 3 persons, 134.7ms
Speed: 13.9ms preprocess, 134.7ms inference, 332.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 133.5ms
Speed: 0.0ms preprocess, 133.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 130.0ms
Speed: 5.6ms preprocess, 130.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 135.3ms
Speed: 3.0ms preprocess, 135.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 persons, 167.6ms
Speed: 11.6ms preprocess, 167.6ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 persons, 132.7ms
Speed: 12.5ms preprocess, 132.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 159.0ms
Speed: 4.0ms preprocess, 159.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 132.2ms
Spe